# Introduction to Variables 

- A tensorflow variable is the recommended way to represent shared and presistent state a program manipulates.
- This notebook covers how to create, update, and manage the instances of tf.Variable in Tensorflow
- `tf.Variable` class can be used to create and track the variables
- _ops_ update and operate on the values
- high-level modules like Keras use variables to store trainable parameters of a model

In [2]:
import tensorflow as tf

In [4]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

A variable looks and acts like a tensor, and, in fact, is a data structure backed by a tf.Tensor. Like tensors, they have a dtype and a shape, and can be exported to NumPy.

In [5]:
print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())

Shape:  (2, 2)
DType:  <dtype: 'float32'>
As NumPy:  [[1. 2.]
 [3. 4.]]


In [6]:
print("A variable:", my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.math.argmax(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: ", tf.reshape(my_variable, [1,4]))

A variable: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Index of highest value: tf.Tensor([1 1], shape=(2,), dtype=int64)

Copying and reshaping:  tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


`assign` method could be called on a variable to reassign value: no new memory location is assigned the previous memory location is used, which is why `assign` only takes in a tensor of similary size and shape

In [14]:
a = tf.Variable(tf.constant([1., 4.]))

#assigning a new value to the variable
a.assign(tf.constant([3.,6.]))

try:
    a.assign(tf.constant([1., 3., 5.]))
except Exception as e:
    print(f"{type(e).__name__} : {e}")

ValueError : Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (2,), and the assigned value shape (3,) are incompatible.


If you use a variable like a tensor in operations, you will usually operate on the backing tensor.

Creating new variables from existing variables duplicates the backing tensors. Two variables will not share the same memory.

In [23]:
a = tf.Variable([1., 9.])

#create new variable based on a
b = tf.Variable(a) # a backing tensor
a.assign([2, 1])

print(f"a: {a.numpy()}")
print(f"b: {b.numpy()}\n")

print(f"{a.assign_sub([-1, -1]).numpy()}")
print(f"{b.assign_add(a).numpy()}")

a: [2. 1.]
b: [1. 9.]

[3. 2.]
[ 4. 11.]


## Life Cycle, Naming, and Watching

In [28]:
# Create a and b; they will have the same name but will be backed by
# different tensors.
a = tf.Variable(my_tensor, name="Mark")
# A new variable with the same name, but different value
# Note that the scalar add is broadcast
b = tf.Variable(my_tensor + 1, name="Mark")

# These are elementwise-unequal, despite having the same name
print(a == b)

tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


- Variable names are saved and preserved when loading the model
- While variables are important for differentiation, some variables will not need differentiation, for such vairables gradient could be turned off by setting the `trainable` parameter to `False`

In [35]:
step_counter = tf.Variable(1, trainable=False)
print(step_counter.numpy())

1


In [36]:
step_counter.assign_add(1).numpy()

2

If you run this notebook on different backends with and without a GPU you will see different logging. Note that logging device placement must be turned on at the start of the session.

In [43]:
with tf.device('CPU:100'):

  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [44]:
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  # Element-wise multiply
  k = a * b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)
